In [2]:
import random
import datetime
from dateutil.relativedelta import relativedelta
from faker import Faker
random.seed(49)
fake = Faker(seed=49,locale="es")

1- Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota,fecha). Se pide resolver utilizando PySpark:

A. Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.

B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón,promedio).

C. El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un segundo RDD alumnos con registros (padron, nombre y apellido).

In [173]:
cantidad_alumnos = 20
padrones = [ x for x in range(1, cantidad_alumnos+1) ]
materias = ("ALGEBRA I", "ANALISIS I", "ALGEBRA II", "ANALISIS II", "PROBABILIDAD", "ALGORITMOS I", "ALGORITMOS II")
notas = list(range(1,11))
fechas = ("01-02-2019", "01-07-2019", "01-02-2018", "01-07-2018", "01-12-2018", "01-02-2017", "01-07-2017", "01-12-2017")
def alumno_random(): return tuple([random.choice(x) for x in (padrones, materias, notas, fechas)])

cantidad_registros = 50
notas = { alumno_random() for a in range(cantidad_registros) }

rdd_notas = sc.parallelize(notas)

def es_fecha_ultimos_anios(fecha, anios=2):
    hoy = datetime.datetime.today()
    d,m,y = fecha.split("-")
    fecha = datetime.date(int(y), int(m), int(d))
    anios_desde_fecha = relativedelta(hoy,fecha).years
    return anios_desde_fecha <= anios

es_aprobado = lambda x: x > 3

ejercicio_a = rdd_notas\
    .filter(lambda x: es_aprobado(x[2]))\
    .filter(lambda x: es_fecha_ultimos_anios(x[3]))\
    .map(lambda x: (x[0], 0))\
    .groupByKey()\
    .count()

print("{} alumnos de {} aprobaron al menos 1 materia en los últimos 2 años".format(ejercicio_a, cantidad_alumnos))

ejercicio_b = rdd_notas\
    .map(lambda x: (x[0], (x[2], 1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+1))\
    .map(lambda x: (x[0], x[1][0]/x[1][1]))
# pide el RDD, no ejecuto ninguna acción
print(ejercicio_b)

nombres = [(x, fake.first_name(),fake.last_name()) for x in padrones]
rdd_nombres = sc.parallelize(nombres)

padron_mejor_promedio = ejercicio_b.max(lambda x: x[1])[0]
ejercicio_c = rdd_nombres.filter(lambda x: x[0]==padron_mejor_promedio).collect()[0]
print("{}, {} (padron {}) es el alumno con mejor promedio".format(ejercicio_c[2], ejercicio_c[1], ejercicio_c[0]))

17 alumnos de 20 aprobaron al menos 1 materia en los últimos 2 años
PythonRDD[261] at RDD at PythonRDD.scala:53
Agudo, Nerea (padron 5) es el alumno con mejor promedio


---

2- Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de  producto, precio de venta) y en otro RDD detalle de los productos con (código de producto, descripción del producto, categoría). Se pide resolver utilizando PySpark:

A. Cuál es el producto más vendido.

B. Cuál es la categoría de productos más vendida.

C. Cuál es el top5 de productos más vendidos generando un RDD con (código de producto, descripción, cantidad de ventas)

D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando para este análisis solo los productos que cuenten con al menos 50 ventas en el último año.

E. Idem anterior, pero calculando la categoría de productos que registró mayor variación de precios en el último año.

In [166]:
cantidad_ventas = 20000
ids_productos = list(range(1, 11))
categorias = ["Autos", "Motos", "Lanchas", "Aviones"]
productos = [ (a,fake.street_name(),random.choice(categorias)) for a in ids_productos ]
def fecha_ultimos_anios(anios=3): return fake.date_between("-5y").strftime("%d-%m-%Y")
ventas = [(fecha_ultimos_anios(),random.choice(ids_productos), random.randint(1,100)) for x in range(cantidad_ventas)]
rdd_ventas = sc.parallelize(ventas)
rdd_productos = sc.parallelize(productos)

In [167]:
# A
codigo_mas_vendido = rdd_ventas.map(lambda x: (x[1], 0))\
    .reduceByKey(lambda x,y: x+1)\
    .max(lambda x: x[1])[0]
producto_mas_vendido = rdd_productos.filter(lambda x: x[0]==codigo_mas_vendido).collect()[0]
print("El producto más vendido es {}".format(producto_mas_vendido))

El producto más vendido es (5, 'Urbanización Luisa Lillo', 'Lanchas')


In [168]:
# C
top_5 = rdd_ventas.map(lambda x: (x[1], 0))\
    .reduceByKey(lambda x,y: x+1)\
    .top(5, lambda x: x[1])
rdd_top_5 = sc.parallelize(top_5, 1)
answer = rdd_top_5.join(rdd_productos.map(lambda x: x[:2]))\
    .map(lambda x: (x[0],x[1][1],x[1][0]))\
    .collect()
print(answer)

[(9, 'Cuesta Jose Carlos España', 227), (1, 'Glorieta Josep Palacio', 217), (10, 'Rambla Gonzalo Mas', 207), (5, 'Urbanización Luisa Lillo', 233), (6, 'Acceso de Esperanza Arnal', 226)]


In [179]:
# D
productos_al_menos_50_ventas_ultimo_anio = rdd_ventas.filter(lambda x: es_fecha_ultimos_anios(x[0], 1))\
    .map(lambda x: (x[1], 0))\
    .reduceByKey(lambda x,y: x+1)\
    .filter(lambda x: x[1] > 50)

In [188]:
ventas_productos_al_menos_50_ventas_ultimo_anio = productos_al_menos_50_ventas_ultimo_anio.join(rdd_ventas.map(lambda x: (x[1], (x[0],x[2]))))

In [189]:
ventas_productos_al_menos_50_ventas_ultimo_anio.aggregateByKey()

(1, (80, ('20-06-2018', 42)))